In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Carregar os dados do arquivo Excel
file_path = 'latencies by service.xlsx'
df = pd.read_excel(file_path)

# Conversão das latências em uma lista de números e reset do índice
df_exploded = df['latencies'].str.split(',', expand=True).stack().astype(int).reset_index(name='latency')
df_exploded.drop('level_1', axis=1, inplace=True)

# Usando Isolation Forest para detectar anomalias
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
anomaly_labels = iso_forest.fit_predict(df_exploded[['latency']])
df_exploded['anomaly'] = np.where(anomaly_labels == -1, 1, 0)

# Engenharia de features
window_size = 5
df_exploded['rolling_mean'] = df_exploded['latency'].rolling(window=window_size).mean()
df_exploded['rolling_std'] = df_exploded['latency'].rolling(window=window_size).std()

# Removendo os valores NaN gerados pela função rolling
df_exploded.dropna(inplace=True)

# Normalização dos dados
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_exploded[['latency', 'rolling_mean', 'rolling_std']])

# Formatação dos dados para entrada na LSTM ([samples, time steps, features])
X = np.array([scaled_features[i-window_size:i] for i in range(window_size, len(scaled_features))])
y = df_exploded['anomaly'][window_size:].values

# Divisão em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Construção do modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(window_size, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))  # 1 unidade para classificação binária

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento do modelo LSTM
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Avaliação do modelo
performance = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {performance[0]}, Test Accuracy: {performance[1]}')

# Salvar o modelo treinado para uso posterior
model.save('lstm_anomaly_detection_model.h5')

# Exibir as anomalias detectadas pelo Isolation Forest
anomalies_detected = df_exploded[df_exploded['anomaly'] == 1]
print("Anomalias detectadas pelo Isolation Forest:")
print(anomalies_detected)

# Salvar as anomalias em um novo arquivo Excel
anomalies_detected.to_excel('detected_anomalies.xlsx', index=False)

# Visualização de anomalias
plt.figure(figsize=(14, 6))
plt.plot(df_exploded.index, df_exploded['latency'], 'b-', label='Latências')
plt.scatter(anomalies_detected.index, anomalies_detected['latency'], color='red', label='Anomalias')
plt.xlabel('Índice')
plt.ylabel('Latência')
plt.title('Latências com Anomalias Destacadas')
plt.legend()
plt.show()
